<a href="https://colab.research.google.com/github/OSegun/Deep-Learning-Projects/blob/main/Clothing_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [1]:
!pip install torchmetrics --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 10.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Accuracy, Precision, Recall
from torchvision import datasets
from torchvision.transforms import transforms

In [5]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 26.4M/26.4M [00:01<00:00, 14.7MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 305kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.62MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.18MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [6]:
classes = train_data.classes
num_classes = len(classes)

In [12]:
print("The Fashions to predicts are:\n ", ", ".join(classes))
print("The number of classes to predict is:", num_classes)

The Fashions to predicts are:
  T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
The number of classes to predict is: 10


In [16]:
image_size = train_data[0][0].shape[1]
image_size

28

In [23]:
class ImageClassifier(nn.Module):

  def __init__(self, num_classes, image_size):
    super(ImageClassifier, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.flatten = nn.Flatten()
    self.layer = nn.Linear(16 *(image_size//2)**2, num_classes)


  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.flatten(x)
    x = self.layer(x)
    return x

In [29]:
train_dataloader = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True
)

In [30]:
def train_model(optimizer, model, epochs):
  processed = 0
  criterion = nn.CrossEntropyLoss()
  for epoch in range(epochs):
    running_loss = 0
    processed = 0
    for feature, label in train_dataloader:
      optimizer.zero_grad()
      output = model(feature)
      loss = criterion(output, label)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      processed += len(label)
    print(f"Epoch {epoch}, Loss: {running_loss/processed}")
  train_loss = running_loss /len(train_dataloader)

In [31]:
model = ImageClassifier(num_classes=num_classes, image_size=image_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_model(
    optimizer=optimizer,
    model=model,
    epochs=5
)

Epoch 0, Loss: 0.04089180020809872
Epoch 1, Loss: 0.033682949545248024
Epoch 2, Loss: 0.03177553457958505
Epoch 3, Loss: 0.030830264964715266
Epoch 4, Loss: 0.030341342069184127


In [32]:
test_dataloader = DataLoader(
    test_data,
    batch_size=10,
    shuffle=False
)

In [39]:
accuracy = Accuracy(task="multiclass", num_classes=num_classes)
precision = Precision(task="multiclass", num_classes=num_classes, average=None)
recall = Recall(task="multiclass", num_classes=num_classes, average=None)

In [40]:
model.eval()
predicted = []
for i, (feature, label) in enumerate(test_dataloader):
  output = model.forward(feature.reshape(-1,1,image_size, image_size))
  pred = torch.argmax(output, dim=-1)
  predicted.extend(pred.tolist())
  accuracy(pred,label)
  precision(pred,label)
  recall(pred, label)

In [41]:
accuracy_val = accuracy.compute().item()
precision_val = precision.compute().tolist()
recall_val = recall.compute().tolist()
print("Accuracy: ", accuracy_val)
print("Precision per class: ", precision_val)
print("Recall per class: ", recall_val)

Accuracy:  0.8794999718666077
Precision per class:  [0.7798408269882202, 0.9751737713813782, 0.8130939602851868, 0.8793274164199829, 0.7558348178863525, 0.9746450185775757, 0.739847719669342, 0.9572301506996155, 0.9817997813224792, 0.9330143332481384]
Recall per class:  [0.8820000290870667, 0.9819999933242798, 0.7699999809265137, 0.8889999985694885, 0.8420000076293945, 0.9610000252723694, 0.5830000042915344, 0.9399999976158142, 0.9710000157356262, 0.9750000238418579]
